##### Import package

In [1]:
# conda create -n sss python=3.8 && conda activate sss
# conda install -y -c bioconda gffutils jupyter tqdm cyvcf2 pathlib2 pandarallel pysam liftover pybedtools

import os
import re
import numpy as np
import pandas as pd
# from Bio.Seq import Seq
# from liftover import get_lifter
from pathlib2 import Path
from pandarallel import pandarallel
from tqdm import tqdm
import gffutils
import pysam
from cyvcf2 import VCF

### Logging setup
from logging import getLogger, config
import yaml
parent_directory = os.path.dirname(os.path.dirname('__file__'))
config_path: str = os.path.join(parent_directory, '../../../config/logging.yaml')
with open(config_path, 'r') as f:
    config.dictConfig(yaml.safe_load(f))
logger = getLogger(__name__)

########   Initialize and setup pandas methods   ########
os.environ['JOBLIB_TEMP_FOLDER'] = '/tmp' 
pandarallel.initialize(nb_workers=3, progress_bar=False, verbose=0, use_memory_fs=False) 
tqdm.pandas()

import sys
try: 
    __file__
    sys.path.append(os.path.join(os.path.dirname('__file__')))
except NameError:
    Path().resolve()
    sys.path.append(os.path.join(Path().resolve(), '../../../'))

from libs import utils, preprocess, variantfilter, posparser, splaiparser
# from libs import predeffect, scoring
from libs import anno_spliceai, anno_clinvar
from libs.deco import print_filtering_count
from libs import predeffect
from libs.scoring import Scoring

gencode_gff = '../../../Resources/05_GENCODE_v43lift37/gencode.v43lift37.annotation.sort.gff3.gz'

try:
    db_anno_gencode = '../../../Resources/06_gffutilsdb/gencode.v43lift37.annotation.gtf.db'
    db_anno_intron = '../../../Resources/06_gffutilsdb/gencode.v43lift37.annotation.intron.gtf.db'
    db = gffutils.FeatureDB(db_anno_gencode)
    db_intron = gffutils.FeatureDB(db_anno_intron)
except ValueError:
    db_anno_gencode = '/resources/DBs/gencode.v43lift37.annotation.gtf.db'
    db_anno_intron = '/resources/DBs/gencode.v43lift37.annotation.intron.gtf.db'
    db = gffutils.FeatureDB(db_anno_gencode)
    db_intron = gffutils.FeatureDB(db_anno_intron)

## Thresholds configuration
thresholds_SpliceAI_parser: dict = {
    'TH_min_sALDL': 0.02, 'TH_max_sALDL': 0.2, 
    'TH_min_sAGDG': 0.01, 'TH_max_sAGDG': 0.05,
    'TH_min_GExon': 25, 'TH_max_GExon': 500,
    'TH_sAG': 0.2, 'TH_sDG': 0.2
    }

In [33]:
!for raw_vcf in splai_vep_vcfs/hgmd_dm/all_DM_chr*.splai.vep.vcf

splai_vep_vcfs/hgmd_dm/all_DM_chr1.splai.vep.vcf
splai_vep_vcfs/hgmd_dm/all_DM_chr10.splai.vep.vcf
splai_vep_vcfs/hgmd_dm/all_DM_chr11.splai.vep.vcf
splai_vep_vcfs/hgmd_dm/all_DM_chr12.splai.vep.vcf
splai_vep_vcfs/hgmd_dm/all_DM_chr13.splai.vep.vcf
splai_vep_vcfs/hgmd_dm/all_DM_chr14.splai.vep.vcf
splai_vep_vcfs/hgmd_dm/all_DM_chr15.splai.vep.vcf
splai_vep_vcfs/hgmd_dm/all_DM_chr16.splai.vep.vcf
splai_vep_vcfs/hgmd_dm/all_DM_chr17.splai.vep.vcf
splai_vep_vcfs/hgmd_dm/all_DM_chr18.splai.vep.vcf
splai_vep_vcfs/hgmd_dm/all_DM_chr19.splai.vep.vcf
splai_vep_vcfs/hgmd_dm/all_DM_chr2.splai.vep.vcf
splai_vep_vcfs/hgmd_dm/all_DM_chr20.splai.vep.vcf
splai_vep_vcfs/hgmd_dm/all_DM_chr21.splai.vep.vcf
splai_vep_vcfs/hgmd_dm/all_DM_chr22.splai.vep.vcf
splai_vep_vcfs/hgmd_dm/all_DM_chr3.splai.vep.vcf
splai_vep_vcfs/hgmd_dm/all_DM_chr4.splai.vep.vcf
splai_vep_vcfs/hgmd_dm/all_DM_chr5.splai.vep.vcf
splai_vep_vcfs/hgmd_dm/all_DM_chr6.splai.vep.vcf
splai_vep_vcfs/hgmd_dm/all_DM_chr7.splai.vep.vcf
splai_v

In [2]:
## Parse VCF to simple input table
chr_list = [str(i) for i in range(1, 23)] + ['X', 'Y']

for chr in chr_list:
    print(f"Processing chr{chr}")
    raw_vcf: str = f"splai_vep_vcfs/hgmd_dm/all_DM_chr{chr}.splai.vep.nondel.vcf"

    vcf = VCF(raw_vcf)
    header = vcf.header_iter()
    for h in header:
        try:
            h['ID']
        except KeyError:
            continue
        else:
            if h['ID'] == 'CSQ':
                vep_cols_list = h['Description'].split('Format: ')[1].rstrip('"').split('|')
            elif h['ID'] == 'SpliceAI':
                splai_cols_list = h['Description'].split('Format: ')[1].rstrip('"').split('|')
            else:
                pass

    vepidx: dict = {col: i for i, col in enumerate(vep_cols_list)}
    splaidx: dict = {col: i for i, col in enumerate(splai_cols_list)}

    cols = [
        'CHROM', 'POS', 'REF', 'ALT', 'GeneSymbol', 'SymbolSource', 'HGNC_ID', 
        'ENST', 'HGVSc', 'Consequence', 'EXON', 'INTRON', 'Strand',
        'DS_AG', 'DS_AL', 'DS_DG', 'DS_DL', 'DP_AG', 'DP_AL', 'DP_DG', 'DP_DL', 'MaxSpliceAI'
    ]

    # print(vepidx)

    df: pd.DataFrame = pd.DataFrame(columns=cols)
    for v in VCF(raw_vcf):
        vep: list = v.INFO.get('CSQ').split('|')

        # Get SpliceAI scores
        if v.INFO.get('SpliceAI'):
            splai: list = v.INFO.get('SpliceAI').split(',')[0].split('|')
        else:
            splai = ['NA'] * len(splai_cols_list)

        # Get HGVSc from VEP
        try:
            hgvsc = re.search('(?<=:).*',vep[vepidx['HGVSc']])[0]
        except TypeError:
            hgvsc = "NA"

        # Convert strand to +/- 
        strand = lambda s: '+' if s == '1' else '-'

        # Get max SpliceAI scores
        ds_ag: float = splai[splaidx['DS_AG']]
        ds_al: float = splai[splaidx['DS_AL']]
        ds_dg: float = splai[splaidx['DS_DG']]
        ds_dl: float = splai[splaidx['DS_DL']]
        if splai[splaidx['DP_AG']] == 'NA':
            maxsplai: str = "NA"
        maxsplai: float = max(ds_ag, ds_al, ds_dg, ds_dl)

        # Add df row
        df = pd.concat([df, pd.DataFrame([[
            v.CHROM, v.POS, v.REF, v.ALT[0], 
            vep[vepidx['SYMBOL']], vep[vepidx['SYMBOL_SOURCE']], vep[vepidx['HGNC_ID']], 
            vep[vepidx['Feature']], hgvsc, vep[vepidx['Consequence']], 
            vep[vepidx['EXON']], vep[vepidx['INTRON']],
            strand(vep[vepidx['STRAND']]), 
            ds_ag, ds_al, ds_dg, ds_dl,
            splai[splaidx['DP_AG']], splai[splaidx['DP_AL']], 
            splai[splaidx['DP_DG']], splai[splaidx['DP_DL']],
            maxsplai
        ]], columns=cols)], ignore_index=True)

        # if hgvsc == "NA":
        #     logger.warning(f"[{v.CHROM}:{v.POS}] HGVSc not found")
        # if maxsplai == "NA":
        #     logger.warning(f"[{v.CHROM}:{v.POS}] SpliceAI scores not found")

    # ALLELE|SYMBOL|DS_AG|DS_AL|DS_DG|DS_DL|DP_AG|DP_AL|DP_DG|DP_DL
    # CHROM, POS, REF, ALT, GeneSymbol, NCBI_ID, ENST, ExonIntronNumbers, FLAGS, SYMBOL_SOURCE|

    df.to_pickle(f"splai_vep_vcfs/hgmd_dm/all_DM_chr{chr}.splai.vep.nondel.vcf.pkl")


Processing chr1
Processing chr2
Processing chr3
Processing chr4
Processing chr5
Processing chr6
Processing chr7
Processing chr8
Processing chr9
Processing chr10
Processing chr11
Processing chr12
Processing chr13
Processing chr14
Processing chr15
Processing chr16
Processing chr17
Processing chr18
Processing chr19
Processing chr20
Processing chr21
Processing chr22
Processing chrX
Processing chrY


In [6]:
# df = pd.read_pickle('splai_vep_vcfs/hgmd_dm/all_DM_chr1.splai.vep.pkl')
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))

20701
20687


In [4]:
#### Very slow process ####
# Annotate ENST Full ID for fetching variant information from GENCODE database

for chr in chr_list:
	logger.info(f"chr{chr}")
	df = pd.read_pickle(f'splai_vep_vcfs/hgmd_dm/all_DM_chr{chr}.splai.vep.nondel.vcf.pkl')
	print(len(df))
	df.drop_duplicates(inplace=True)
	df['ENST_Full'] = df.progress_apply(posparser.fetch_enst_full, db=db, axis=1)
	df.to_pickle(f'splai_vep_vcfs/hgmd_dm/all_DM_chr{chr}.splai.vep.nondel.vcf.enst.pkl')

2024/10/20 08:09:49 [INFO   ] (__main__) - chr1
20588


100%|██████████| 20574/20574 [07:13<00:00, 47.47it/s]


2024/10/20 08:17:02 [INFO   ] (__main__) - chr2
21002


100%|██████████| 20983/20983 [06:52<00:00, 50.85it/s]  

2024/10/20 08:23:55 [INFO   ] (__main__) - chr3


...

2024/10/20 08:54:37 [INFO   ] (__main__) - chrX


31081


100%|██████████| 31016/31016 [04:19<00:00, 119.59it/s]


2024/10/20 08:58:56 [INFO   ] (__main__) - chrY
116


100%|██████████| 116/116 [00:00<00:00, 744.95it/s]


## ここから解析

In [ ]:
for chr in chr_list:
    df = pd.read_pickle(f'splai_vep_vcfs/hgmd_dm/all_DM_chr{chr}.splai.vep.nondel.vcf.enst.pkl')

    logger.info('Classify "Canonical" splice site or "Non-canonical" splice site...')
    df = posparser.classifying_canonical(df)

    logger.info('Calculate the distance to the nearest splice site in intron variant...')
    df['IntronDist'] = df.progress_apply(
        posparser.signed_distance_to_exon_boundary, 
        db=db, db_intron=db_intron, axis=1)

    tbx_anno = pysam.TabixFile(gencode_gff)
    df['exon_loc'] = df.progress_apply(
        posparser.calc_exon_loc, tabixfile=tbx_anno, enstcolname='ENST', axis=1)
    df = pd.concat([df, df['exon_loc'].str.split(':', expand=True)], axis=1)
    df.rename(columns={0: 'ex_up_dist', 1: 'ex_down_dist'}, inplace=True)
    df.drop(columns=['exon_loc'], inplace=True)

    #2-2. Select minimum distance from upstream distance and downstream distance
    df['exon_pos'] = df.parallel_apply(posparser.select_exon_pos, axis=1)
    #2-3. Relative exon location
    df['prc_exon_loc'] = df.parallel_apply(posparser.calc_prc_exon_loc, axis=1)

    #2-4. Decision exonic splice sites (1 nt in acceptor site or 3 nts on Donor site)
    df['exon_splice_site'] = df.parallel_apply(posparser.extract_splicing_region, axis=1)

    #3.   Additional Splicing information
    logger.info('Annotating splicing information...')
    #3-1. Annotate splicing type ('Exonic Acceptor' etc.)
    df['SpliceType'] = df.parallel_apply(posparser.select_donor_acceptor, axis=1)

    #5.   Annotate ClinVar varaints interpretations
    logger.info('Annotating ClinVar varaints interpretations...')
    clinvar_file = '../../../Resources/03_ClinVar/variant_summary.snv.grch37.germline.criteria.sort.bed.gz'
    tbx_clinvar = pysam.TabixFile(clinvar_file)
    df['clinvar_same_pos'] = df.progress_apply(
        anno_clinvar.anno_same_pos_vars, tabixfile=tbx_clinvar, axis=1)
    df['clinvar_same_motif'] = df.progress_apply(
        anno_clinvar.anno_same_motif_vars, tabixfile=tbx_clinvar, axis=1)

    logger.info('Parsing SpliceAI results...')
    logger.info('Annotating Exon/Intron position information...')
    df['ExInt_INFO'] = df.progress_apply(
        splaiparser.calc_exint_info, db=db, db_intron=db_intron, axis=1)

    #6-3. Predict splicing effects
    df['Pseudoexon'] = df.progress_apply(
        splaiparser.pseudoexon_activation,
        thresholds=thresholds_SpliceAI_parser, 
        db_intron=db_intron,
        axis=1)

    df['Part_IntRet'] = df.parallel_apply(
        splaiparser.partial_intron_retention,
        thresholds=thresholds_SpliceAI_parser, 
        axis=1)

    df['Part_ExDel'] = df.parallel_apply(
        splaiparser.partial_exon_deletion,
        thresholds=thresholds_SpliceAI_parser, 
        axis=1)

    df['Exon_skipping'] = df.parallel_apply(
        splaiparser.exon_skipping, 
        thresholds=thresholds_SpliceAI_parser, 
        axis=1)
                                            
    df['Int_Retention'] = df.parallel_apply(
        splaiparser.intron_retention, 
        thresholds=thresholds_SpliceAI_parser, 
        axis=1)

    df['multiexs'] = df.parallel_apply(
        splaiparser.multi_exon_skipping, 
        thresholds=thresholds_SpliceAI_parser, 
        axis=1)

    # df = pd.read_pickle('splai_vep_vcfs/hgmd_dm/all_DM_chr1.splai.vep.enst.introndist.exintinfo.splicing2.pkl')
    #7.   Annotate aberrant splicing size (bp)
    logger.info('Annotating aberrant splicing size (bp)...')
    #7-1. Annotate size of 
    df['Size_Part_ExDel'] = df.parallel_apply(
        splaiparser.anno_partial_exon_del_size, 
        thresholds=thresholds_SpliceAI_parser, 
        axis=1)

    #7-3. Annotate size of partial intron retention
    df['Size_Part_IntRet'] = df.parallel_apply(
        splaiparser.anno_partial_intron_retention_size, 
        thresholds=thresholds_SpliceAI_parser,
        axis=1)

    #7-2. Annotate size of pseudoexon
    df['Size_pseudoexon'] = df.parallel_apply(
        splaiparser.anno_gained_exon_size, 
        thresholds=thresholds_SpliceAI_parser, 
        axis=1)

    #7-4. Annotate size of intron retention
    df['Size_IntRet'] = df.parallel_apply(
        splaiparser.anno_intron_retention_size, 
        thresholds=thresholds_SpliceAI_parser,
        axis=1)

    #7-5. Annotate size of exon skipping
    df['Size_skipped_exon'] = df.parallel_apply(
        splaiparser.anno_skipped_exon_size, 
        thresholds=thresholds_SpliceAI_parser,
        axis=1)

    df['variant_id'] = df['CHROM'].astype(str) + '-' \
        + df['POS'].astype(str) + '-' + df['REF'] + '-' + df['ALT']

    #8.   Evaluate splicing effects
    logger.info('Predicting CDS change...')
    #8-1. Predict CDS change
    df['CDS_Length'] = df.progress_apply(predeffect.calc_cds_len, db=db, axis=1)
    df['is_10%_truncation'] = df.progress_apply(predeffect.calc_cds_len_shorten, axis=1)

    #8-2. Determine if the gene is included in eLoFs genes
    df['is_eLoF'] = df.parallel_apply(predeffect.elofs_judge, axis=1)

    #8-3. Determine causing NMD or not
    df['is_NMD_at_Canon'] = df.parallel_apply(predeffect.nmd_judge, axis=1)

    #8-4. Frame check
    # Covert to str (Cannot predict splicing event) to np.nan


    cannot_predict: str = 'Cannot predict splicing event'
    df['Size_Part_ExDel'] = df['Size_Part_ExDel'].replace(cannot_predict, np.nan)
    df['Size_Part_IntRet'] = df['Size_Part_IntRet'].replace(cannot_predict, np.nan)
    df['Size_pseudoexon'] = df['Size_pseudoexon'].replace(cannot_predict, np.nan)
    df['Size_IntRet'] = df['Size_IntRet'].replace(cannot_predict, np.nan)
    df['Size_skipped_exon'] = df['Size_skipped_exon'].replace(cannot_predict, np.nan)

    df['is_Frameshift_Part_ExDel'] = df['Size_Part_ExDel'].parallel_apply(
        predeffect.frame_check)
    df['is_Frameshift_Part_IntRet'] = df['Size_Part_IntRet'].parallel_apply(
        predeffect.frame_check)
    df['is_Frameshift_pseudoexon'] = df['Size_pseudoexon'].parallel_apply(
        predeffect.frame_check)
    df['is_Frameshift_IntRet'] = df['Size_IntRet'].parallel_apply(
        predeffect.frame_check)
    df['is_Frameshift_skipped_exon'] = df['Size_skipped_exon'].parallel_apply(
        predeffect.frame_check)
    df['is_Frameshift'] = df[['is_Frameshift_Part_ExDel', 
                            'is_Frameshift_Part_IntRet', 
                            'is_Frameshift_pseudoexon', 
                            'is_Frameshift_IntRet', 
                            'is_Frameshift_skipped_exon'
                            ]].any(axis=1)

    #9.   CCRs
    logger.info('Annotating CCRs info...')
    #9-1. Annotate truncated regions 
    df['skipped_region'] = df.parallel_apply(
        splaiparser.anno_skipped_regions, axis=1)
    df['deleted_region'] = df.parallel_apply(
        splaiparser.anno_deleted_regions, 
        thresholds=thresholds_SpliceAI_parser, axis=1)

    #9-2. Intersect with CCRs
    logger.info('Annotate CCR score')
    df = predeffect.anno_ccr_score(df)

    df.to_pickle(f'splai_vep_vcfs/hgmd_dm/all_DM_chr{chr}.splai.vep.nondel.vcf.enst.prescore.pkl')
    logger.info(f"Comleted chr{chr}")

In [ ]:
df.to_pickle(f'splai_vep_vcfs/hgmd_dm/all_DM_chr{chr}.splai.vep.nondel.vcf.enst.prescore.pkl')

In [9]:
df: pd.DataFrame = pd.concat([pd.read_pickle(f'splai_vep_vcfs/hgmd_dm/all_DM_chr{chr}.splai.vep.nondel.vcf.enst.prescore.pkl') for chr in chr_list])

#10.  Scoring
sccore_ths = {'clinvar_same_pos': 2,     
                'clinvar_same_motif': 1,
                'clinvar_else': 0,
                'non_canon_splai_lte_0.1_outside': -3,
                'non_canon_splai_lte_0.1_other': -2,
                'non_canon_splai_bet_0.1_0.2': 1,
                'non_canon_splai_gte_0.2': 2,
                'canon_strong': 6, 
                'canon_moderate': 5, 
                'frameshift_nmd_eloF': 7, 
                'frameshift_nmd_not_eloF': 3,
                'canon_splai_lte_0.1': -3,
                'canon_splai_bet_0.1_0.2': -1,
                'canon_splai_gte_0.2': 0}
scoring = Scoring(ths=sccore_ths)

logger.info('Annotating Screening scores...')
df.rename(columns={'MaxSpliceAI': 'maxsplai'}, inplace=True)
df['insilico_screening'] = df.parallel_apply(scoring.insilico_screening, axis=1)
df['clinvar_screening'] = df.parallel_apply(scoring.clinvar_screening, axis=1)
df['PriorityScore'] = df.parallel_apply(scoring.calc_priority_score, axis=1)

2024/10/21 02:15:04 [INFO   ] (__main__) - Annotating Screening scores...


In [53]:
df.to_pickle('splai_vep_vcfs/hgmd_dm/all_DM_chr1-22.splai.vep.nondel.vcf.enst.scored.pkl')

In [54]:
df = pd.read_pickle('splai_vep_vcfs/hgmd_dm/all_DM_chr1-22.splai.vep.nondel.vcf.enst.scored.pkl')
df = df[['variant_id', 'CHROM', 'POS', 'HGVSc', 'maxsplai', 'clinvar_screening', 'insilico_screening', 'PriorityScore']]
df['variant_id2'] = df['CHROM'].astype(str) + '-' + df['POS'].astype(str) + '-' + df['HGVSc']
df = df.loc[df['PriorityScore'] != 'Not available']

In [86]:
all_mut_default_colnames: list = [
    "disase", "gene", "chrom", "genename", "gdbid", "omimid", "amino", 
    "deletion", "insertion", "codon", "codonAff", "descr", "refseq", "hgvs", 
    "hgvsAll", "dbsnp", "chromosome", "startCoord", "endCoord", 
    "expected_inheritance", "gnomad_AC", "gnomad_AF", "gnomad_AN", "tag", 
    "dmsupport", "rankscore", "mutype", "author", "title", "fullname", 
    "allname", "vol", "page", "year", "pmid", "pmidAll", "reftag", "comments", 
    "acc_num", "new_date", "base", "clinvarID", "clinvar_clnsig"
]
allmut: pd.DataFrame = pd.read_csv(
    'allmut.csv', sep=';', encoding='cp1252', names=all_mut_default_colnames, 
    skiprows=1,low_memory=False)

allmut = allmut[
    ["gene", "genename", "mutype", "clinvar_clnsig", "tag",
     "refseq", "hgvs", "hgvsAll", "chromosome", "startCoord", "endCoord", 
     "amino", "deletion", "insertion", "expected_inheritance", "gnomad_AF"]]

# Drop non-numeric values in 'startCoord'
allmut = allmut.dropna(subset=['startCoord'])

# Drop duplicates in 'chrom', 'startCoord', and 'endCoord'
allmut = allmut.drop_duplicates(subset=['chromosome', 'startCoord', 'endCoord'])

# Extract tag == "DM" from allmut
allmut_dm = allmut[allmut.tag == "DM"].copy()
print(f"A total of {len(allmut_dm)} DM mutations are found in allmut.")

allmut_dm['startCoord'] = allmut_dm['startCoord'].astype(int)
allmut_dm = allmut_dm.rename(columns={'chromosome': 'CHROM', 'startCoord': 'POS_hg38'})
# allmut_dm['variant_id2'] = allmut_dm['chromosome'].astype(str) + '-' \
# 	+ allmut_dm['startCoord'].astype(str) + '-' + allmut_dm['hgvs']

A total of 253018 DM mutations are found in allmut.


In [87]:
# Fillna with empty string in "gnomad_AF" colmun in allmut_dm
# Extratct MAF 0 from allmut_dm
allmut_dm['gnomad_AF'].fillna(0, inplace=True)
allmut_dm_maf0 = allmut_dm[allmut_dm['gnomad_AF'] == 0].copy()
print(f"A total of {len(allmut_dm_maf0)} DM mutations are found in allmut with MAF 0.")

# Extract non-deletion or non-insertion from allmut_dm
allmut_dm_maf0_snv = allmut_dm_maf0[(allmut_dm_maf0['deletion'].isnull()) & (allmut_dm_maf0['insertion'].isnull())]
print(f"A total of {len(allmut_dm_maf0_snv)} DM mutations are found in allmut with MAF 0 and non-deletion or non-insertion.")

# Extract the mutation type from the mutype column
splice_mutations = allmut_dm_maf0_snv[allmut_dm_maf0_snv["mutype"].str.contains("splice")].copy()
non_splice_mutations = allmut_dm_maf0_snv[~allmut_dm_maf0_snv["mutype"].str.contains("splice")]
print(f"Splicing_DM: {len(splice_mutations)}, Non-splicing_DM: {len(non_splice_mutations)}")

A total of 231981 DM mutations are found in allmut with MAF 0.
A total of 154107 DM mutations are found in allmut with MAF 0 and non-deletion or non-insertion.
Splicing_DM: 22178, Non-splicing_DM: 131929


In [88]:
# Convert startCoord to hg19
from liftover import get_lifter

def _liftover_to_hg19(chrom, pos):
    converter = get_lifter('hg38', 'hg19')
    result = converter.query(chrom, pos)
    if result:
        return result[0]
    else:
        return None
    
def anno_hg19_pos(row):
    converted = _liftover_to_hg19(row['CHROM'], row['POS_hg38'])
    return converted[1]

non_splice_mutations['POS_hg19'] = non_splice_mutations.apply(anno_hg19_pos, axis=1)


KeyboardInterrupt: 

In [ ]:
non_splice_mutations.to_csv('non_splice_mutations.csv', index=False)

In [ ]:
non_splice_mutations['variant_id2'] = non_splice_mutations['chromosome'].astype(str) + '-' \
	+ non_splice_mutations['startCoord'].astype(str) + '-' + non_splice_mutations['hgvs']

non_splice_mutations_variants: list = non_splice_mutations['variant_id2'].tolist()
df['is_non_splice_DM'] = df['variant_id2'].isin(non_splice_mutations_variants)

In [72]:
for row in df.itertuples():
	if row.variant_id2 in non_splice_mutations_variants:
		# df.at[row.Index, 'is_non_splice_DM'] = True
		print(row.variant_id2)

KeyboardInterrupt: 

In [64]:
len(df['variant_id2'])

250733

In [34]:
len(allmut)

510804

In [2]:
def _liftover_to_hg19(chrom, pos):
    converter = get_lifter('hg38', 'hg19')
    result = converter.query(chrom, pos)
    if result:
        return result[0]
    else:
        return None
    
def anno_vars_id(row):
    variant_id = f'{row["CHROM"]}:{row["POS_hg19"]}-{row["REF"]}-{row["ALT"]}'
    return variant_id

def anno_hg19_pos(row):
    converted = _liftover_to_hg19(row['CHROM'], row['POS_hg38'])
    return converted[1]

def remove_dot_ver(x):
    if x == '.':
        pass
    else:
        return re.match(r'[a-zA-Z_]+\d+', x).group()
    
def remove_non_canon(x):
    if x in enst_set:
        return True
    else:
        pass

def create_refalt(row, nt):
    if row['Strand'] == '+':
        return row[nt]
    elif row['Strand'] == '-':
        return str(Seq(row[nt]).complement())
    else:
        pass

In [3]:
canonlist = '/Github/Projects/DeNovo/data/CanonicalTranscripts_COMP/CanonicalTranscripts.exoncount.tsv'
refseq = '/work/CanonicalTrasncripts/gencode.v43lift37.metadata.RefSeq.gz'

df_canon = pd.read_table(canonlist, header=0)
df_canon = df_canon[(df_canon['ENST'] != 'ENST00000649912')
                    & (df_canon['ENST'] != 'ENST00000609375')]
df_enst = df_canon.drop_duplicates(subset='ENST')
enst_set = set(df_enst['ENST'])
df_refseq = pd.read_table(refseq, header=None, 
                          names=['ENST_refseq', 'RefSeq_RNA', 'RefSeq_Pro'])
df_refseq.fillna(value='.', inplace=True)

df_refseq['ENST_refseq'] = df_refseq['ENST_refseq'].apply(remove_dot_ver)
df_refseq['RefSeq_RNA'] = df_refseq['RefSeq_RNA'].apply(remove_dot_ver)
df_refseq['RefSeq_Pro'] = df_refseq['RefSeq_Pro'].apply(remove_dot_ver)

df_refseq['is_Canonical'] =  df_refseq['ENST_refseq'].apply(remove_non_canon)
df_refseq_canon = df_refseq[df_refseq['is_Canonical'] == True].copy()

##### Generate VCF

In [4]:
# Loading HGMD splicing variants list (tsv)
hgmd_file = '/Github/MyProjects/DeNovo/data/ValidationData/Positive/allmut.trim.colfixed.maf0.tsv.txt'
df = pd.read_table(hgmd_file, header=0)

# Pre-processing
df = df.dropna(subset=['startCoord'])
df['startCoord'] = df['startCoord'].astype(int)
df['refseq'] = df['refseq'].apply(remove_dot_ver)

# Extract REF and ALT from HGVS descriptions
sr_alt_nt = df['hgvs'].str[-1:].rename('alt_nt')
sr_ref_nt = df['hgvs'].str[-3:-2].rename('ref_nt')
df = pd.concat([df, sr_ref_nt, sr_alt_nt], axis=1)

# Insert cols as VCF
df.loc[:,'ID'] = '.'
df.loc[:,'QUAL'] = '.'
df.loc[:,'FILTER'] = '.'
df.loc[:,'INFO'] = '.'

# Annotate ENST IDs
df = pd.merge(df, df_enst, how='left', 
              left_on='gene', right_on='GeneSymbol')

# Rename cols for downstream processing
df = df.rename(columns={'CHROM': 'Chr',
                        'chromosome': 'CHROM', 
                        'startCoord': 'POS_hg38'})

# Create REF and ALT columns
df['REF'] = df.apply(create_refalt, nt='ref_nt', axis=1)
df['ALT'] = df.apply(create_refalt, nt='alt_nt', axis=1)

In [5]:
# Liftover to hg19
df['POS_hg19'] = df.apply(anno_hg19_pos, axis=1)

In [ ]:
df2 = df.copy()
df2 = df2.astype({'POS_hg19': str})

# Annotate variant IDs
df2['variant_id'] = df2.apply(anno_vars_id, axis=1)

# Extract columns for VCF
df_19 = df2[['CHROM', 'POS_hg19', 
             'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO']]
df_38 = df2[['CHROM', 'POS_hg38', 
             'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO']]

In [265]:
df_19 = df_19.dropna(subset=['REF'])
df_38 = df_38.dropna(subset=['REF'])

In [266]:
# Output as VCF
df_19.to_csv('./patho.hg19.vcf', sep='\t', index=False, header=False)
df_38.to_csv('./patho.hg38.vcf', sep='\t', index=False, header=False)

##### Concatenate header and variant list

In [267]:
!cat ../header_for_VCF.tsv ./patho.hg19.vcf > patho.hg19.header.vcf
!cat ../header_for_VCF.tsv ./patho.hg38.vcf > patho.hg38.header.vcf

In [263]:
df[['Strand', 'ref_nt', 'REF', 'alt_nt', 'ALT']]

,Strand,ref_nt,REF,alt_nt,ALT
0,+,A,A,G,G
1,-,T,A,G,C
2,-,G,C,A,T
3,+,A,A,G,G
4,-,A,T,G,C
...,...,...,...,...,...
10553,NaN,G,None,A,None
10554,+,G,G,A,A
10555,-,G,C,A,T
10556,-,G,C,A,T


In [277]:
df2 =df2[['gene', 'gdbid', 'omimid', 'amino', 'codon', 'codonAff', 'descr',
       'refseq', 'hgvs', 'CHROM', 'POS_hg38', 'endCoord', 
       'expected_inheritance', 'dmsupport', 'mutype', 'acc_num', 'new_date', 
       'clinvarID', 'clinvar_clnsig', 'Chr', 'Start', 'End', 'Strand', 
       'GeneSymbol', 'HGNC_ID', 'ENSG', 'ENST', 'GeneType', 'Tag', 
       'REF', 'ALT', 'POS_hg19', 'variant_id']]

In [278]:
df2.to_csv('./patho2.tsv', sep='\t', index=False)

In [ ]:
#解除後の賃料はどうなっているか